In [95]:
#Loeme andmed sisse ja vvlime esialu ainult 5 riiki
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np
from sklearn import preprocessing
covidDeath = pd.read_csv('time_series_covid19_deaths_global.csv', sep=",")
#covidDeath.head(10)

osa = covidDeath.loc[(covidDeath['Country/Region'] == 'Estonia') |  (covidDeath['Country/Region'] == 'Lithuania') | (covidDeath['Country/Region'] == 'Finland')| (covidDeath['Country/Region'] == 'Sweden') | (covidDeath['Country/Region'] == 'Lithuania') | (covidDeath['Country/Region'] == 'New Zealand')]
osa.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21
113,NaN,Estonia,58.595300,25.013600,0,0,0,0,0,0,...,213,221,226,229,234,241,244,251,258,261
117,NaN,Finland,61.924110,25.748151,0,0,0,0,0,0,...,546,550,556,561,561,561,561,565,576,576
169,NaN,Lithuania,55.169400,23.881300,0,0,0,0,0,0,...,1269,1347,1422,1458,1589,1614,1643,1950,1996,2036
195,NaN,New Zealand,-40.900600,174.886000,0,0,0,0,0,0,...,25,25,25,25,25,25,25,25,25,25
236,NaN,Sweden,60.128161,18.643501,0,0,0,0,0,0,...,8279,8484,8727,8727,8727,8727,8727,8727,8985,8985


In [96]:
##Koristame hetkel ebavajaliku info
vahendatud = osa.copy()
vahendatud = vahendatud.drop('Province/State', axis = 'columns')
vahendatud =  vahendatud.drop('Lat', axis = 'columns')
vahendatud =  vahendatud.drop('Long', axis = 'columns')
vahendatud = vahendatud.set_index('Country/Region')
vahendatud

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Estonia,0,0,0,0,0,0,0,0,0,0,...,213,221,226,229,234,241,244,251,258,261
Finland,0,0,0,0,0,0,0,0,0,0,...,546,550,556,561,561,561,561,565,576,576
Lithuania,0,0,0,0,0,0,0,0,0,0,...,1269,1347,1422,1458,1589,1614,1643,1950,1996,2036
New Zealand,0,0,0,0,0,0,0,0,0,0,...,25,25,25,25,25,25,25,25,25,25
Sweden,0,0,0,0,0,0,0,0,0,0,...,8279,8484,8727,8727,8727,8727,8727,8727,8985,8985


In [97]:
#Teostame standarrdiseerimise, sisuliselt z skooridde arvutuse. valisin standardiseerimise 
#normaliseerimise üle, sest meil riigiti sammud ja sisuliselt ühikud seega erinevad
tulbad = vahendatud.columns
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(vahendatud)
scaled_df = pd.DataFrame(scaled_df, columns=tulbad)
ind = ['Est', 'Fin', 'Lit', 'NZe','Swe']
scaled_df['index'] = ind
scaled_df = scaled_df.set_index('index')
scaled_df

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21
index,,,,,,,,,,,,,,,,,,,,,
Est,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.591165,-0.593055,-0.595042,-0.597250,-0.605093,-0.605242,-0.606539,-0.625670,-0.624511,-0.626395
Fin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.484951,-0.490600,-0.495121,-0.496648,-0.505823,-0.508044,-0.510210,-0.529957,-0.530391,-0.533128
Lit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.254341,-0.242404,-0.232906,-0.224840,-0.193744,-0.188200,-0.181415,-0.107784,-0.110103,-0.100846
NZe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.651130,-0.654092,-0.655902,-0.659066,-0.668541,-0.670851,-0.673088,-0.694559,-0.693474,-0.696271
Swe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.981587,1.980152,1.978971,1.977805,1.973201,1.972337,1.971253,1.957971,1.958480,1.956640


In [78]:
##Tegin kümnepäevase Eesti näidisamplituudi
eesti_vordlus = np.array([scaled_df['12/28/20']['Est'], scaled_df['12/29/20']['Est'], scaled_df['12/30/20']['Est'],scaled_df['12/31/20']['Est'],scaled_df['1/1/21']['Est'],scaled_df['1/2/21']['Est'],scaled_df['1/3/21']['Est'],scaled_df['1/4/21']['Est'],scaled_df['1/5/21']['Est'],scaled_df['1/6/21']['Est']])
eesti_vordlus

array([-0.59116524, -0.59305521, -0.5950418 , -0.59725035, -0.60509308,
       -0.60524205, -0.60653924, -0.62567048, -0.62451139, -0.62639488])

In [99]:
# teeme kõikidest päevadest kümnepäevased intervvallid, millega meile 
#huvipakkuvat ajavvahemikku võrrelda
#EEesti oma katseks
eesti_ = {}
eesti={}
eestiList = []
for a in range (0,351):
    eestiList.append(scaled_df.iloc[0, a])
    vahe = 0
    for b in range (a, 351):
        vahe = vahe +1
        if vahe < 11:
            eesti_.setdefault(a,[]).append(scaled_df.iloc[0, b])
        else:
            break

for k in eesti_.keys():
    if len(eesti_[k]) == 10:
        eesti.setdefault(k,eesti_[k])



In [93]:
##SOOME 10-PÄEVASED INTERVALLID, ARVUTASIN KÕIGI INTERVALLIDE KAUGUSE EESTI 10-PÄEVASE JUPIGA 
## TULEMUSEKS 10 KÕIGE VÄIKSEMAT DISTANTSI
soome_ = {}
soome={}
for a in range (0,351):
    vahe = 0
    for b in range (a, 351):
        vahe = vahe +1
        if vahe < 11:
            soome_.setdefault(a,[]).append(scaled_df.iloc[1, b])
        else:
            break

for k in soome_.keys():
    if len(soome_[k]) == 10:
        soome.setdefault(k,soome_[k])

kaugused = {}
for k in soome.keys():
    distance, path = fastdtw(eesti_vordlus, soome[k], dist=euclidean)
    kaugused.setdefault(distance,[]).append(k)
for s in sorted(kaugused)[:10]:
    print(str(s) + ' ' + str(kaugused[s]))

0.9850900822670076 [341]
1.0298036247821027 [340]
1.0699637272213005 [48]
1.0699637272213007 [49]
1.0736447575775925 [339]
1.095862876408919 [50]
1.1169558191374198 [51]
1.1188693255431343 [338]
1.1325304885147793 [52]
1.1387241301048636 [53]


In [92]:
#SAMA LEEDUGA
leedu_ = {}
leedu={}
for a in range (0,351):
    vahe = 0
    for b in range (a, 351):
        vahe = vahe +1
        if vahe < 11:
            leedu_.setdefault(a,[]).append(scaled_df.iloc[2, b])
        else:
            break

for k in leedu_.keys():
    if len(leedu_[k]) == 10:
        leedu.setdefault(k,leedu_[k])

kaugused = {}
for k in leedu.keys():
    distance, path = fastdtw(eesti_vordlus, leedu[k], dist=euclidean)
    kaugused.setdefault(distance,[]).append(k)
for s in sorted(kaugused)[:10]:
    print(str(s) + ' ' + str(kaugused[s]))

0.7321810704937531 [103]
0.7325888862771356 [102]
0.7327962133841994 [104]
0.7332533232807871 [106]
0.7332892766788268 [105]
0.7336336465539125 [107]
0.7354770775947634 [101]
0.7354947054423914 [108]
0.7373837189061629 [100]
0.7389748143015199 [109]


In [90]:
#SAMA UUS-MEREMAAAGA
meremaa = {}
meremaa_={}
for a in range (0,351):
    vahe = 0
    for b in range (a, 351):
        vahe = vahe +1
        if vahe < 11:
            meremaa_.setdefault(a,[]).append(scaled_df.iloc[3, b])
        else:
            break

for k in meremaa_.keys():
    if len(meremaa_[k]) == 10:
        meremaa.setdefault(k,meremaa_[k])

kaugused = {}
for k in meremaa.keys():
    distance, path = fastdtw(eesti_vordlus, meremaa[k], dist=euclidean)
    kaugused.setdefault(distance,[]).append(k)
for s in sorted(kaugused)[:10]:
    print(str(s) + ' ' + str(kaugused[s]))

0.05415331501733789 [317]
0.05476928774390366 [316]
0.05957756309542417 [318]
0.0600627814390986 [315]
0.06299191142607152 [314]
0.06396740828325342 [319]
0.07175140694603732 [313]
0.07252758019939187 [320]
0.07950580959731068 [321]
0.08378140820101376 [312]


In [91]:
#SAMA ROOTSIGA
rootsi = {}
rootsi_={}
for a in range (0,351):
    vahe = 0
    for b in range (a, 351):
        vahe = vahe +1
        if vahe < 11:
            rootsi_.setdefault(a,[]).append(scaled_df.iloc[4, b])
        else:
            break

for k in rootsi_.keys():
    if len(rootsi_[k]) == 10:
        rootsi.setdefault(k,rootsi_[k])

kaugused = {}
for k in rootsi.keys():
    distance, path = fastdtw(eesti_vordlus, rootsi[k], dist=euclidean)
    kaugused.setdefault(distance,[]).append(k)
for s in sorted(kaugused)[:10]:
    print(str(s) + ' ' + str(kaugused[s]))

6.0699637272213005 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
8.0699637272213 [39]
10.0699637272213 [40]
12.0699637272213 [41]
14.0699637272213 [42]
16.0699637272213 [43]
18.0699637272213 [44]
20.069963727221303 [45]
22.069963727221303 [46]
24.069963727221303 [47]
